#### MARKETPULSE STOCK MARKET TREND ANALYSIS

In [ ]:
%reset -f


In [ ]:
# INSTALLING BASIC LIBRARIES
%pip install pandas
%pip install numpy 
%pip install scipy 
%pip install matplotlib seaborn scikit-learn ta
%pip install seaborn

In [ ]:
# IMPORTING BASIC LIBRARIES
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import ta                                        # for technical analysis indicators


In [ ]:
# loading the data set

stock_prices = pd.read_csv(r"C:\Users\TRIPLE D.TECH\Downloads\Amdari\First Project\stock_prices.csv")
stock_P_indicators = pd.read_csv(r"C:\Users\TRIPLE D.TECH\Downloads\Amdari\First Project\stock_prices_with_indicators.csv")
comp_Info = pd.read_csv(r"C:\Users\TRIPLE D.TECH\Downloads\Amdari\First Project\company_info.csv")
market_indices = pd.read_csv(r"C:\Users\TRIPLE D.TECH\Downloads\Amdari\First Project\market_indices.csv")  




In [ ]:
# Verifying data loading

datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

# Loop through each dataset and display info
for name, df in datasets.items():
    print(f"===== {name} =====")
    print("Shape:", df.shape, "\n")
    
    print("Info:")
    df.info()
    print("\nHead:")
    display(df.head()) 
    print("\n" + "="*60 + "\n")




#### *INSIGHT:
##### The following data sets: Stock prices, stock prices with indicators, company info, market Indices have a mismatch in the data types of the ticker,date, ipo_date.
##### - Ticker appear as object rather than strings 
##### - Date also appears as object rather than a date datatype
##### - ipo_date is also an object rather than a date datatype

In [ ]:
# Verifying missing values in the data sets
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

for name, df in datasets.items():
    print(f"===== {name} =====")
    print("\nMissing Values:")
    display((df.isnull().sum() / len(df) * 100).map(lambda x: f"{x:.2f}%"))


print("\n" + "="*60 + "\n")

##### *INSIGHT: This shows that all other data sets are free of missing values except for the stock prices indicators data set

In [ ]:
# Checking for duplicates
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

for name, df in datasets.items():
    print(f"===== {name} =====")
    print("\nDuplicate Records:")
    duplicates = df.duplicated().sum()  # counts duplicate rows
    print(f"Total duplicate rows: {duplicates}")
    
    print("\n" + "="*60 + "\n")

#### *INSIGHT: There are no duplicates in any of the data sets.

In [ ]:
# Verifying date ranges and continuity in time-series data

# datasets dictionary
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

# Date columns for each dataset
date_columns = {
    'stock_prices': 'date',
    'stock_P_indicators': 'date',
    'comp_Info': 'ipo_date',        # special column name
    'market_indices': 'date'
}

# Loop through each dataset
for name, df in datasets.items():
    print(f"===== {name} =====")
    
    # Get the correct date column for this dataset
    date_col = date_columns.get(name)
    if date_col not in df.columns:
        print(f"No '{date_col}' column found in this dataset. Skipping continuity check.\n")
        print("="*60 + "\n")
        continue
    
    # Ensure date column is datetime
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Start and end dates
    start_date = df[date_col].min()
    end_date = df[date_col].max()
    print(f"Start date: {start_date}")
    print(f"End date: {end_date}\n")
    
    # Check continuity
    df_sorted = df.sort_values(date_col)
    full_range = pd.date_range(start=start_date, end=end_date, freq='D')  # Change 'D' if not daily
    missing_dates = full_range.difference(df_sorted[date_col])
    
    print(f"Total missing dates: {len(missing_dates)}")
    if len(missing_dates) > 0:
        print("Missing dates:")
        print(missing_dates)
    else:
        print("No missing dates. Continuity is fine.")
    
    print("\n" + "="*60 + "\n")



============================================================================================
##### * Summary findings & Deliverables: Data quality report documenting:
--------------------------------------------------------------------------------------------
●	Missing value percentages per column = Done
●	Data type corrections needed = Ensuring Ticker and date columns to be corrected to reflect their true data types (i.e string and date type)
●	Duplicate records count = No duplicate records found
●	Date range coverage per stock = Missing dates discovered.

---------------------------------------------------------------------------------------------

 #### * Plan for handling issues documented
 --------------------------------------------------------------------------------------------
 ●	Data type corrections needed = Ticker and date columns would be corrected to ensure they reflect their true data types (i.e string and date type)
 
 ●  Missing dates discovered: - i.e either applying Forward fill (ffill) which carries last observation forward (common for financial data).
         or                   -  Backward fill (bfill) method which fills with next available value.

### Data cleaning

##### 1. Converting the date and ticker column to their respective data type: object(string) & datetime

In [ ]:
# datasets dictionary
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

# Date columns for each dataset
date_columns = {
    'stock_prices': 'date',
    'stock_P_indicators': 'date',
    'comp_Info': 'ipo_date',
    'market_indices': 'date'
}

# Looping through each dataset
for name, df in datasets.items():
    print(f"Processing dataset: {name}")
    
    # Convert 'ticker' column to string if it exists
    if 'ticker' in df.columns:
        df['ticker'] = df['ticker'].astype(str)
        print(" - Converted 'ticker' to string")
    
    # Convert date column to datetime
    date_col = date_columns.get(name)
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')  # 'coerce' converts invalid dates to NaT
        print(f" - Converted '{date_col}' to datetime")
    
    print("\n")


In [ ]:
# Confirming the convertion of these columns

# datasets dictionary
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

# Loop through each dataset and show the first 5 rows
for name, df in datasets.items():
    print(f"===== {name} =====")
    display(df.head())
    
    print("\nColumn Data Types:")
    display(df.dtypes)
    
    print("\n" + "="*60 + "\n")


##### Handling Missing values in our data sets

In [ ]:

datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

for name, df in datasets.items():
    # Detect time-series: any datetime column
    is_time_series = any(df[col].dtype == 'datetime64[ns]' for col in df.columns)

    if is_time_series:
        # Forward fill for time-series data
        df.fillna(method='ffill', inplace=True)
        
        # Backward fill for any remaining NaNs
        df.fillna(method='bfill', inplace=True)
        print(f"{name}: Time-series detected, applied ffill then bfill")
    else:
        # Non-time-series: backward fill or drop if minimal
        df.fillna(method='bfill', inplace=True)
        if df.isna().sum().sum() > 0:
            df.dropna(inplace=True)
            print(f"{name}: Non-time-series, bfill applied, remaining NaNs dropped")
        else:
            print(f"{name}: Non-time-series, bfill applied, no NaNs remain")



In [ ]:
# Confirming missing values 
datasets = {
    'stock_prices': stock_prices,
    'stock_P_indicators': stock_P_indicators,
    'comp_Info': comp_Info,
    'market_indices': market_indices
}

for name, df in datasets.items():
    print(f"===== {name} =====")
    print("\nMissing Values:")
    display((df.isnull().sum() / len(df) * 100).map(lambda x: f"{x:.2f}%"))


print("\n" + "="*60 + "\n")

In [ ]:
stock_prices.head(2)

### Exploratory Data Analysis





##### 1. Price Trend Analysis:

In [ ]:
# Merging stock price data set with company information on the "ticker" column to include sector data
stock_prices = stock_prices.merge(comp_Info[['ticker', 'sector']], on='ticker', how='left')


In [ ]:
stock_prices.head(2)

In [ ]:
# Stock selection

sample_stocks = (
    stock_prices.groupby('sector')['ticker']
    .first()  # first stock in each sector
    .head(5)  # pick only 5 sectors                           # Picking one stock from each of 5 different sectors
    .values
)

print(sample_stocks)


In [ ]:
# selecting only the rows in stock_prices that belong to the 5 chosen tickers
trend_data = stock_prices[stock_prices['ticker'].isin(sample_stocks)]



In [ ]:
trend_data[['sector', 'ticker']].value_counts()

In [ ]:
# Plotting closing price trends

plt.figure(figsize=(10, 5))
sns.lineplot(data=trend_data, x='date', y='close', hue='ticker')
plt.title('Closing Price Trends for Selected Stocks')
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.legend(title='Ticker')

plt.show()

##### *INSIGHT: 
###### -Stock with ticker STK001(Technology) shows the highest upward trend followed by the stock STK009(Finance).
######  - The ticker STK013(Consumer) exhibits a sideway trend. same as the Healthcare  stock(STK005)
###### - The Energy stock(STK017) however exhibits more of a downward trend over the years.

In [ ]:
# Calculating  the basic statistics: mean, median, std dev of returns

# Compute daily returns for each stock
trend_data['daily_return'] = trend_data.groupby('ticker')['close'].pct_change()

# calculate basic statistics for each stock
return_stats = trend_data.groupby(['sector', 'ticker'])['daily_return'].agg(['mean', 'median', 'std']).reset_index()

# Convert mean and std to percentages for readability
return_stats['mean'] = return_stats['mean'] * 100
return_stats['median'] = return_stats['median'] * 100
return_stats['std'] = return_stats['std'] * 100

# Round results
return_stats = return_stats.round(2)

display(return_stats)


##### *INSIGHT:
--------------------------------------------------------------------------------------------------------------
###### mean : Shows the average daily return (%) i.e shows the typical daily gain/loss.

###### median:shows the middle daily return (%). 

###### Standard deviation of daily returns (%): i.e measures the volatility — how much the stock price fluctuates daily. Higher values mean more risk.

--------------------------------------------------------------------------------------------------------------
###### - Highest average daily return: Technology (STK001) and Finance (STK009).

###### - Most volatile stock: Energy (STK017) → highest std dev at 2.96%.

###### - Least volatile stock: Consumer (STK013) → lowest std dev at 1.89%.

###### -Healthcare (STK005) had slightly negative median daily returns → suggests more frequent small losses than gains.

------------------------------------------------------------------------------------------------------------------------

#### 2. Sector Comparison

In [ ]:
# Visualizing sector performance with box plots

plt.figure(figsize=(10,6))

# Box plot of daily returns by sector
sns.boxplot(data=trend_data, x='sector', y='daily_return')

plt.title('Sector Performance: Distribution of Daily Returns')
plt.xlabel('Sector')
plt.ylabel('Daily Return')
plt.xticks(rotation=45)  # rotate x labels for readability
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


#### *INSIGHT:Outliers represent stocks whose returns are far from the typical range in that sector.
##### NB: The Outliers in each sector reveals that some stocks perform better giving higher returns, 
#####  while some perfom badly giving negative returns. The Technology sector for examples as seen above
#####  has most of its stocks giving returns of 2.5%, but some few stocks give returns of above 7.5%, and negative retunrs of -7.5%. 



#### 3.	Volume Analysis

In [ ]:
# Plotting volume patterns over time

# Picking the first stock from each sector
sample_stocks_v = stock_prices.groupby('sector')['ticker'].first().values

# Filter data for plotting
volume_data = stock_prices[stock_prices['ticker'].isin(sample_stocks_v)]

plt.figure(figsize=(10, 7))
sns.lineplot(data=volume_data, x='date', y='volume', hue='ticker', legend=True)  
plt.title('Trading Volume Patterns Over Time for All Tickers')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.show()

##### *INSIGHT: This reveals a large clustering around  1.0 to 1.4 million shares implying stocks typically trade around this volume.

In [ ]:

# stock per sector
sample_stocks = stock_prices.groupby('sector')['ticker'].first().values
stock_data = stock_prices[stock_prices['ticker'].isin(sample_stocks)].copy()

# Calculate daily return (%)
stock_data['daily_return'] = stock_data.groupby('ticker')['close'].pct_change() * 100

# Identify high-volume days (top 5% per stock)
stock_data['high_volume'] = stock_data.groupby('ticker')['volume'].transform(
    lambda x: x > x.quantile(0.95)
)


plt.figure(figsize=(10,6))
sns.scatterplot(data=stock_data, x='volume', y='daily_return', hue='ticker', style='high_volume', s=100)
plt.title('Relationship Between Volume and Price Changes')
plt.xlabel('Trading Volume')
plt.ylabel('Daily Return (%)')
plt.show()


###### *INSIGHT: - Most points are clustered around low to moderate trading volumes and small daily returns, implying the “normal” trading days, and small price changes with the trading volume.

In [ ]:
# Calculating the correlation between volume and price movement

# Grouping by ticker and calculating the correlation between volume and daily_return
correlations = stock_data.groupby('ticker').apply(
    lambda x: x['volume'].corr(x['daily_return'])
).reset_index()

correlations.columns = ['ticker', 'volume_return_corr']
display(correlations)


##### This generally shows a very weak correlation between the volume and price change

#### 4. Technical Indicator Exploration:



In [ ]:
stock_P_indicators.columns

In [ ]:
# Visualizing RSI patterns and identifying overbought/oversold conditions

# Choosing a sample ticker to visualize 
chosenRSI_ticker = stock_P_indicators['ticker'].unique()[0]
ticker_df = stock_P_indicators[stock_P_indicators['ticker'] == chosenRSI_ticker]

# Plot RSI pattern
plt.figure(figsize=(10,5))
plt.plot(ticker_df['date'], ticker_df['rsi_14'], label='RSI (14)', color='blue')

# Adding overbought and oversold lines
plt.axhline(70, color='red', linestyle='--', label='Overbought (70)')
plt.axhline(30, color='green', linestyle='--', label='Oversold (30)')

# Titles and labels
plt.title(f'RSI Pattern for {chosenRSI_ticker}')
plt.xlabel('Date')
plt.ylabel('RSI (14)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()



#### *INSIGHT: 
###### * The RSI (blue line) above 70 suggests the stock might be overbought — i.e
######    too many investors have been buying it, pushing the price up to a level that may be unsustainable in the near term.

###### *  The RSI below 30 suggests the stock might be oversold — i.e 
######     too many investors have been selling it, pushing the price below its fair value or creating an opportunity for a potential rebound.

In [ ]:

# Pick one ticker to visualize
chosenRSI_ticker = stock_P_indicators['ticker'].unique()[0]
ticker_df = stock_P_indicators[stock_P_indicators['ticker'] == chosenRSI_ticker]

# Calculating MACD and Signal
ticker_df['MACD'] = ticker_df['close'].ewm(span=12, adjust=False).mean() - ticker_df['close'].ewm(span=26, adjust=False).mean()
ticker_df['Signal'] = ticker_df['MACD'].ewm(span=9, adjust=False).mean()

# Identifying crossovers using the 'macd' and 'macd_signal' columns
ticker_df['Bullish'] = (ticker_df['MACD'] > ticker_df['Signal']) & (ticker_df['MACD'].shift(1) <= ticker_df['Signal'].shift(1))
ticker_df['Bearish'] = (ticker_df['MACD'] < ticker_df['Signal']) & (ticker_df['MACD'].shift(1) >= ticker_df['Signal'].shift(1))

In [ ]:
# Plotting MACD crossovers and their relationship to price trends


# Plot
plt.figure(figsize=(10,5))
plt.plot(ticker_df['date'], ticker_df['close'], label='Close Price', color='blue')
plt.scatter(ticker_df[ticker_df['Bullish']]['date'], ticker_df[ticker_df['Bullish']]['close'], color='green', marker='^', label='Bullish')
plt.scatter(ticker_df[ticker_df['Bearish']]['date'], ticker_df[ticker_df['Bearish']]['close'], color='red', marker='v', label='Bearish')
plt.title('MACD crossovers and their relationship to price trends')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(alpha=0.4)
plt.show()


#### 5.	Correlation Analysis:



In [ ]:
stock_P_indicators.columns

In [ ]:

# Selecting some numerical columns of interest
Selected_features = ['open', 'high', 'low', 'close','volume', 'rsi_14', 'macd', 'macd_signal']

# Computing the correlation matrix
corr = stock_P_indicators[Selected_features].corr()

# Ploting the heatmap
plt.figure(figsize=(10,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap of Selected Features')
plt.show()


##### *INSIGHT: 
###### This reveals that every change in one column, for instance ('open', 'high', 'low', 'close),
######    is exactly proportional to changes in the others. For example, if open goes up by 1 unit, high, low, 
###### and close also go up by a fixed amount.